In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [33]:
# Set user agent in headers to mimic a web browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

URL = "https://www.profession.hu/allasok/budapest/1,0,23,data%20engineer%401%401?keywordsearch"
# conducting a request of the stated URL above:
page = requests.get(URL, headers=headers)

In [34]:
# Function to extract job titles and job links from the soup
def extract_job_info_from_result(soup):
    job_info = []
    # for a_job_name in soup.find_all("a", attrs={"data-job-name": True}):
    #     job_title = a_job_name["data-job-name"]
    #     # Get the previous sibling that contains the job link
    #     a_job_link = a_job_name.find_previous("a")
    #     job_link = a_job_link
    #     if a_job_link and "href" in a_job_link.attrs:
    #         job_link = a_job_link["href"]
    #     job_info.append({"job_title": job_title, "job_link": job_link})
    # return job_info
    job_items = soup.find_all("li", class_="card")
    for item in job_items:
        job_title = item.find("h2", class_="highlight").get_text(strip=True)
        company_name = item.find("div", class_="job-card__company-name").a.get_text(
            strip=True
        )
        job_location = item.find("div", class_="job-card__company-address").span[
            "title"
        ]
        job_summary = item.find("div", class_="card-text").get_text(strip=True)
        job_link = item.find("a", class_="btn-primary")["href"]

        job_info.append(
            {
                "job_title": job_title,
                "company_name": company_name,
                "location": job_location,
                "job_summary": job_summary,
                "job_link": job_link,
            }
        )

    return job_info

In [35]:
# Function to scrape the page
def scrape_page(url, headers):
    page = requests.get(url, headers=headers)
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "html.parser")
        job_info = extract_job_info_from_result(soup)
        return pd.DataFrame(job_info)
    else:
        print("Failed to retrieve the page. Status code:", page.status_code)
        return None

In [36]:
# Perform scraping with rate limiting
try:
    # Scrape the initial page
    job_info_df = scrape_page(URL, headers)

    if job_info_df is not None:
        # Wait for a random amount of time between 5 to 10 seconds before making the next request
        time.sleep(random.uniform(5, 10))

        # Print the DataFrame
        print("Job Info DataFrame:")
        print(job_info_df.head())
        print(job_info_df["job_link"].iloc[3])
except Exception as e:
    print("An error occurred:", str(e))


An error occurred: 'NoneType' object has no attribute 'get_text'


In [37]:
job_info_df.head()

,tob_title,company_name,location,job_summary,job_link
0,DATA,UNIQA Raiffeisen Software Service Kft.,1053 Budapest,Adatbázisok és adatáramlási architektúrák terv...,https://www.profession.hu/allas/data-engineer-...
1,data,OTP Bank Nyrt.,"Budapest, 13. kerület",Elsősorban a területünk riporting rendszerét f...,https://www.profession.hu/allas/uzleti-adatmod...
2,data,K&H BANK ZRT. (ICT),"1095 Budapest, Lechner Ödön fasor 9. K. épület",Senior Data Engineerként a következő feladatai...,https://www.profession.hu/allas/senior-data-en...
3,Data,Richter Gedeon Nyrt.,Budapest,"Low-code, no-code Power Platform megoldások fe...",https://www.profession.hu/allas/data-engineer-...
4,Data,Tech People Hungary Kft.,"Budapest, XI. kerület",Ensuring development tasks to support new rele...,https://www.profession.hu/allas/senior-data-en...
